# Tutorial 7: Exact Diagonalization of Transverse Ising Model

## Exact Diagonalization of a XXZ Chain 



The so-called XXZ model is given by 
$$
H=\sum_{i=1}^{L} \left( J \left( S_i^x S_{i+1}^x + S_i^y S_{i+1}^y \right) + \Delta S_i^z S_{i+1}^z \right)
$$

When $\Delta=1$, we have a Heisenberg model.
The code below shows how to construct the Hamiltonian using the method described in class. Here we work in the  $S_z=0$ sector. 
Modify this code fro the XXZ model. 

In [2]:
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import eigsh

def construct_hamiltonian(L, J):
    """
    Constructs the Heisenberg Hamiltonian for a spin-1/2 chain of length L with periodic boundary conditions.
    The Hamiltonian is built in the Sz=0 sector.
    """
    # Generate all possible spin configurations as integers
    states = []
    for state in range(2**L):
        spin_sum = sum(1 if (state >> i) & 1 else -1 for i in range(L))
        if spin_sum == 0:  # Select only states with Sz = 0
            states.append(state)

    # Mapping of states to basis indices
    state_to_index = {state: idx for idx, state in enumerate(states)}
    dim = len(states)

    # Sparse matrix storage for the Hamiltonian
    rows, cols, data = [], [], []

    # Construct the Hamiltonian
    for state in states:
        i = state_to_index[state]

        # Iterate over all pairs of neighboring spins
        for site in range(L):
            # Sites to interact
            spin1 = (state >> site) & 1
            spin2 = (state >> ((site + 1) % L)) & 1

            # ZZ term
            zz = (1 if spin1 == spin2 else -1)
            rows.append(i)
            cols.append(i)
            data.append(J * zz * 0.25)

            # XX and YY terms (off-diagonal)
            if spin1 != spin2:
                # Flip spins
                flipped_state = state ^ (1 << site) ^ (1 << ((site + 1) % L))
                j = state_to_index[flipped_state]
                rows.append(i)
                cols.append(j)
                data.append(J * 0.5 * 0.25)

    # Build sparse Hamiltonian matrix
    H = csr_matrix((data, (rows, cols)), shape=(dim, dim), dtype=np.float64)
    return H, states

def heisenberg_ground_state(L, J=1.0):
    """
    Computes the ground state energy and wavefunction of a spin-1/2 Heisenberg chain of length L
    with periodic boundary conditions, in the Sz=0 symmetry sector.
    """
    H, states = construct_hamiltonian(L, J)

    # Find the lowest eigenvalue and eigenvector using sparse diagonalization
    eigenvalues, eigenvectors = eigsh(H, k=1, which='SA')

    ground_state_energy = eigenvalues[0]
    ground_state_wavefunction = eigenvectors[:, 0]

    return ground_state_energy, ground_state_wavefunction, states

# Parameters
L = 6  # Length of the chain
J = 1.0  # Exchange interaction strength

# Compute ground state
energy, wavefunction, states = heisenberg_ground_state(L, J)

print("Ground State Energy:", energy)
print("Ground State Wavefunction:", wavefunction)

		
		
		
		
		


Ground State Energy: -1.6108143415915814
Ground State Wavefunction: [ 0.01163068 -0.0982008   0.0982008  -0.01163068  0.0982008  -0.66463057
  0.0982008   0.0982008  -0.0982008   0.01163068 -0.01163068  0.0982008
 -0.0982008  -0.0982008   0.66463057 -0.0982008   0.01163068 -0.0982008
  0.0982008  -0.01163068]
